Analyze Risk on a Loan Portfolio using Statistical Tests in Jupyter

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("loan-dataset-1.csv")

In [ ]:
df.head()

,loan_id,principal_loan_amount,request_date,interest_rate,loan_channel,is_approved,is_default,customer_monthly_income,customer_age,customer_gender,customer_credit_score,customer_zip_code
0,70280c1b-4339-4949-ba69-ad2a6cb6b782,8880,2023-03-06,9.0,WEB,True,NaN,6100,30,Male,3,10007
1,8a550559-9749-491b-b6aa-7188bc0a488f,13720,2023-05-10,10.0,MOBILE_APP,True,NaN,5150,39,Female,3,10004
2,5644d27e-e05e-4a40-bef3-64187892338f,2090,2023-02-16,7.5,AGENT,True,NaN,6030,43,Female,3,10003
3,9d8d17ac-8af9-4936-ba5d-8d5ed5e0cfab,6820,2023-02-26,8.5,MOBILE_APP,NaN,NaN,2150,54,Male,0,10004
4,4586ad7a-2231-419b-b8e3-ba03b929b56e,13770,2023-04-13,10.0,AFFILIATE,NaN,NaN,5810,28,Female,2,10007


We need to make sure the data is clean

First, are there at least 10,000 distinct rows

In [ ]:
print("Total Rows: ", df.shape[0])

Total Rows:  15163


Are there any duplicate rows?

In [ ]:
print('There are', df.duplicated().sum(),'duplicate rows in the DF')

There are 163 duplicate rows in the DF


Are there NaN values?

In [ ]:
if df.isna().any().any():
    print('There are NaN values in the DataFrame')
    print(df.isna().any(axis=0)[df.isna().any()], '\n')
else:
    print('There are no NaN values in the DataFrame\n')

There are NaN values in the DataFrame
is_approved    True
is_default     True
dtype: bool 



In [ ]:
print('principal_loan_amount:', df['principal_loan_amount'].min(), 'to', df['principal_loan_amount'].max())
print('request_date:', df['request_date'].min(), 'to', df['request_date'].max())
print('interest_rate:', df['interest_rate'].min(), 'to', df['interest_rate'].max())
print('loan_channel:', df['loan_channel'].unique())
print('is_approved:', df['is_approved'].unique())
print('is_default:', df['is_default'].unique())
print('customer_monthly_income:', df['customer_monthly_income'].min(), 'to', df['customer_monthly_income'].max())
print('customer_age:', df['customer_age'].min(), 'to', df['customer_age'].max())
print('customer_gender:', df['customer_gender'].unique())

principal_loan_amount: 1000 to 15000
request_date: 2023-01-01 to 2023-06-29
interest_rate: 7.5 to 10.0
loan_channel: ['WEB' 'MOBILE_APP' 'AGENT' 'AFFILIATE' 'DIRECT_SELLING'
 'website_revamped' 'ANDROID_V2' 'ANDROID_V3' 'apple_new_v3']
is_approved: [True nan]
is_default: [nan True]
customer_monthly_income: 2000 to 6500
customer_age: 22 to 60
customer_gender: ['Male' 'Female' 'M' 'F']


Turn the blank values in the boolean columns to False

In [ ]:
df = df.fillna(False)
df.head()

,loan_id,principal_loan_amount,request_date,interest_rate,loan_channel,is_approved,is_default,customer_monthly_income,customer_age,customer_gender,customer_credit_score,customer_zip_code
0,70280c1b-4339-4949-ba69-ad2a6cb6b782,8880,2023-03-06,9.0,WEB,True,False,6100,30,Male,3,10007
1,8a550559-9749-491b-b6aa-7188bc0a488f,13720,2023-05-10,10.0,MOBILE_APP,True,False,5150,39,Female,3,10004
2,5644d27e-e05e-4a40-bef3-64187892338f,2090,2023-02-16,7.5,AGENT,True,False,6030,43,Female,3,10003
3,9d8d17ac-8af9-4936-ba5d-8d5ed5e0cfab,6820,2023-02-26,8.5,MOBILE_APP,False,False,2150,54,Male,0,10004
4,4586ad7a-2231-419b-b8e3-ba03b929b56e,13770,2023-04-13,10.0,AFFILIATE,False,False,5810,28,Female,2,10007


The following mapping for loan_channel (case insensitive)

website_revamped -> WEB

In [ ]:
df['loan_channel'] = df['loan_channel'].replace('website_revamped', 'WEB')

everything that contains ANDROID or APPLE  -> MOBILE_APP

In [ ]:
df['loan_channel'] = df['loan_channel'].str.replace("ANDROID", "MOBILE_APP")
df['loan_channel'] = df['loan_channel'].str.replace("APPLE", "MOBILE_APP")

For the gender, risk team does not care which combination you use (either F / M or Female / Male )

In [ ]:
df['customer_gender'] = df['customer_gender'].replace('M', 'MALE')
df['customer_gender'] = df['customer_gender'].replace('F', 'FEMALE')

Evaluate The Impact of Interest Rates on Loan Default Rates
You have the following hypotheses to test:

Null hypothesis (from the operations team) : the interest rate has no impact on the loan default rate

Alternative hypothesis : interest rate does have an impact on the loan default rate.

In [ ]:
median = df['interest_rate'].median()

high_rate = df[df['interest_rate'] > median]
low_rate = df[df['interest_rate'] <= median]

In [ ]:
high_default_rate = high_rate['is_default'].mean()
low_default_rate = low_rate['is_default'].mean()

In [ ]:
from scipy import stats

In [ ]:
t_stat, p_val = stats.ttest_ind(high_rate['is_default'], low_rate['is_default'])

In [ ]:
print("High interest default rate", high_default_rate)
print("Low interest default rate", low_default_rate)
print('Two tailed t-test:', t_stat, 'p-value:', p_val)

High interest default rate 0.03371605613267724
Low interest default rate 0.037205456800330716
Two tailed t-test: -1.1091890264137088 p-value: 0.2673662392421917


We can safely fail to reject the hypothesis. .267 is greater than .05.

Evaluate the Relationship Between Income Level and Loan Approval Rates
You have the following hypotheses to test:

Null hypothesis (from the operations team) : customer income level has no impact on the loan approval rate

Alternative hypothesis : customer income level impacts the loan approval rate

Use logistic regression analysis to evaluate the impact of customer income level on loan approval rates. The customer income levels are separated into these groups:

less than 2500

2500 to 5000

5000 to 7500

more than 7500



In [ ]:
import numpy as np
income_groups = pd.cut(df['customer_monthly_income'], bins=[0, 2500, 5000, 7500, np.inf])
approval_rate = df.groupby(income_groups)['is_approved'].mean()

In [ ]:
import statsmodels.api as sm
X = sm.add_constant(df['customer_monthly_income'])
y = df['is_approved']
model = sm.Logit(y, X).fit()
print(model.summary())

Optimization terminated successfully.
         Current function value: 0.531550
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:            is_approved   No. Observations:                15163
Model:                          Logit   Df Residuals:                    15161
Method:                           MLE   Df Model:                            1
Date:                Sun, 30 Jul 2023   Pseudo R-squ.:               8.483e-06
Time:                        12:31:09   Log-Likelihood:                -8059.9
converged:                       True   LL-Null:                       -8060.0
Covariance Type:            nonrobust   LLR p-value:                    0.7115
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                       1.2677      0.067     19.049      0.000       1.137   

Since interest rate does not impact the default rate, you need to find out possible variable(s) that affect loan default rate.

The risk team suggested the following variable possibilities:

customer_monthly_income

customer_zip_code

In [ ]:
X = df[['customer_monthly_income', 'customer_zip_code']]
print(X)
X = sm.add_constant(X)
print(X)

       customer_monthly_income  customer_zip_code
0                         6100              10007
1                         5150              10004
2                         6030              10003
3                         2150              10004
4                         5810              10007
...                        ...                ...
15158                     3900              10005
15159                     3120              10009
15160                     3120              10007
15161                     6390              10009
15162                     3090              10005

[15163 rows x 2 columns]
       const  customer_monthly_income  customer_zip_code
0        1.0                     6100              10007
1        1.0                     5150              10004
2        1.0                     6030              10003
3        1.0                     2150              10004
4        1.0                     5810              10007
...      ...                    

In [ ]:
y = df['is_default']
model = sm.Logit(y, X).fit(method='bfgs')
print(model.summary())

         Current function value: 0.043598
         Iterations: 35
         Function evaluations: 73
         Gradient evaluations: 56
                           Logit Regression Results                           
Dep. Variable:             is_default   No. Observations:                15163
Model:                          Logit   Df Residuals:                    15160
Method:                           MLE   Df Model:                            2
Date:                Sun, 30 Jul 2023   Pseudo R-squ.:                  0.7184
Time:                        12:31:09   Log-Likelihood:                -661.07
converged:                      False   LL-Null:                       -2347.7
Covariance Type:            nonrobust   LLR p-value:                     0.000
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                    4.762e+04   2426.311     

c:\Users\jayso\OneDrive\Documents\UdemyLabs\env\Lib\site-packages\statsmodels\discrete\discrete_model.py:2383: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
c:\Users\jayso\OneDrive\Documents\UdemyLabs\env\Lib\site-packages\statsmodels\discrete\discrete_model.py:2441: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q * linpred)))
c:\Users\jayso\OneDrive\Documents\UdemyLabs\env\Lib\site-packages\statsmodels\discrete\discrete_model.py:2383: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
c:\Users\jayso\OneDrive\Documents\UdemyLabs\env\Lib\site-packages\statsmodels\discrete\discrete_model.py:2441: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q * linpred)))
c:\Users\jayso\OneDrive\Documents\UdemyLabs\env\Lib\site-packages\statsmodels\discrete\discrete_model.py:2383: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
c:\Users\jayso\OneDrive\Documents\UdemyLa

In [ ]:
from statsmodels.formula.api import ols

In [ ]:
df['is_default'] = df['is_default'].astype(int)
formula = 'is_default ~ C(loan_channel)'
model = ols(formula, data=df).fit()
aov_table = sm.stats.anova_lm(model, typ=2)
print(aov_table)